In [1]:
from sklearn.neural_network import MLPClassifier
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
# graficos incrustados
%matplotlib inline

#Carga de los CSV
BBVA_data = pd.read_csv('train_clientes.csv', header=0)
print("Data Train: ", BBVA_data.shape)
BBVA_test = pd.read_csv('test_clientes.csv', header=0)
print("Data Test:  ", BBVA_test.shape)

Data Train:  (70000, 53)
Data Test:   (30000, 52)


In [2]:
X_train = BBVA_data.drop(['ID_CORRELATIVO', 'ATTRITION', 'RANG_INGRESO', 'FLAG_LIMA_PROVINCIA', 'EDAD', 'ANTIGUEDAD'], axis=1)
print(type(X_train), X_train.shape)
X_predic = BBVA_test.drop(['ID_CORRELATIVO','RANG_INGRESO', 'FLAG_LIMA_PROVINCIA', 'EDAD', 'ANTIGUEDAD'], axis=1)
print(type(X_predic), X_predic.shape)
y_train = BBVA_data['ATTRITION'].values
print(type(y_train), y_train.shape)

<class 'pandas.core.frame.DataFrame'> (70000, 47)
<class 'pandas.core.frame.DataFrame'> (30000, 47)
<class 'numpy.ndarray'> (70000,)


In [3]:
print(X_train.isnull().any().any())
print(X_predic.isnull().any().any())

False
False


In [4]:
def factorizacion_data(datus):
    #Pasando los datos de edad a rango de 10 años
    #EDAD2 = pd.cut(datus['EDAD'], range(0, 150, 10))
    #datus['EDAD'] = EDAD2
    
    #Factorizando la columna EDAD
    #datus['EDAD'], _ = pd.factorize(datus['EDAD'])
    
    #Factorizar las demas columnas de datos categoricos
    tipos = datus.columns.to_series().groupby(datus.dtypes).groups
    ctext = tipos[np.dtype('object')]
    for c in ctext:
        datus[c], _ = pd.factorize(datus[c])
    print(type(datus), datus.shape)
    return datus

X_train = factorizacion_data(X_train)
X_predic = factorizacion_data(X_predic)

<class 'pandas.core.frame.DataFrame'> (70000, 47)
<class 'pandas.core.frame.DataFrame'> (30000, 47)


In [5]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler(copy=True, with_mean=True, with_std=True)
xx_train_std = sc_X.fit_transform(X_train.astype(float))
xx_predic_std = sc_X.transform(X_predic.astype(float))

from sklearn.preprocessing import MinMaxScaler
sc_MM = MinMaxScaler()
xx_train_mm = sc_MM.fit_transform(X_train.astype(float))
xx_predic_mm = sc_MM.transform(X_predic.astype(float))

idd = BBVA_test['ID_CORRELATIVO']
idd.shape

(30000,)

In [7]:
initial = 'quitcolumNULL_imputacion_factorizacion_allVar_'
import csv
def generate_csv(clf, predic, name):
    name = initial + name
    result = clf.predict_proba(predic)
    result = [float("{0:.5f}".format(_)) for _ in list(zip(*result))[1]]
    result = list(zip(list(idd), result))
    result.insert(0, ('ID_CORRELATIVO', 'ATTRITION'))
    with open("{}.csv".format(name), 'w') as resultFile:
       wr = csv.writer(resultFile, dialect='excel')
       for row in result:
           wr.writerow(row)

In [8]:
list_train = [(xx_train_std, xx_predic_std), (xx_train_mm, xx_predic_mm)]

for ixx, train in enumerate(list_train):
    red = MLPClassifier(max_iter=1000000000, hidden_layer_sizes=(100,100), alpha=0.0000000001, solver='sgd', random_state=2)
    red.fit(train[0], y_train)
    name ='{0}_RedNeuronal_{1}'.format(
        'STD2' if ixx == 0 else 'MINMAX2', 'sgd')
    generate_csv(red, train[1], name)
    print("quitcolumNULL///////////////////////////////////////////////{0}".format(name))

quitcolumNULL///////////////////////////////////////////////STD2_RedNeuronal_sgd
quitcolumNULL///////////////////////////////////////////////MINMAX2_RedNeuronal_sgd


In [9]:
list_train = [(xx_train_std, xx_predic_std), (xx_train_mm, xx_predic_mm)]
from sklearn.ensemble import RandomForestClassifier
for ixx, train in enumerate(list_train):
    red =  RandomForestClassifier(n_estimators=250, n_jobs=2, min_samples_split=25,
                                      max_features=0.99, random_state=2)
    red.fit(train[0], y_train)
    name ='{0}_RandomForest_{1}'.format(
        'STD2' if ixx == 0 else 'MINMAX2', 'arboles250_sp25')
    generate_csv(red, train[1], name)
    print("quitcolumNULL///////////////////////////////////////////////{0}".format(name))
    

quitcolumNULL///////////////////////////////////////////////STD2_RandomForest_arboles250_sp25
quitcolumNULL///////////////////////////////////////////////MINMAX2_RandomForest_arboles250_sp25


In [10]:
list_train = [(xx_train_std, xx_predic_std), (xx_train_mm, xx_predic_mm)]
from sklearn.ensemble import RandomForestClassifier
for ixx, train in enumerate(list_train):
    red =  RandomForestClassifier(n_estimators=250, n_jobs=2, min_samples_split=45,
                                      max_features=0.99, random_state=2)
    red.fit(train[0], y_train)
    name ='{0}_RandomForest_{1}'.format(
        'STD2' if ixx == 0 else 'MINMAX2', 'arboles250_sp45')
    generate_csv(red, train[1], name)
    print("quitcolumNULL///////////////////////////////////////////////{0}".format(name))

quitcolumNULL///////////////////////////////////////////////STD2_RandomForest_arboles250_sp45
quitcolumNULL///////////////////////////////////////////////MINMAX2_RandomForest_arboles250_sp45
